In [2]:
import sentencepiece as spm

#Train the tokenizer if needed.

#spm.SentencePieceTrainer.train(input='all_classified_paragraphs_text.txt' , model_prefix='FIN_P2A_32k',split_digits=True, train_extremely_large_corpus=False, vocab_size=32000,input_sentence_size=5000000)

In [1]:
import sentencepiece as spm
import numpy as np
import time, random

In [67]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding,Bidirectional,Concatenate,Flatten
from tensorflow.keras.utils import *
from tensorflow.keras.initializers import *
import tensorflow as tf


In [3]:
sp_input = spm.SentencePieceProcessor(model_file='FIN_P2A_8k.model')
sp_tgt = spm.SentencePieceProcessor(model_file='FIN_P2A_8k.model')

In [4]:
#Use training data

train_raw_loc = 'src-train.txt'
target_raw_loc = 'tgt-train.txt'

In [39]:
with open(train_raw_loc, 'r', encoding='utf-8') as f:
    train_raw_lines = f.read().split('\n')
    train_raw_lines = train_raw_lines[:-1]
with open(target_raw_loc, 'r', encoding='utf-8') as f:
    target_raw_lines = f.read().split('\n')
    target_raw_lines = target_raw_lines[:-1]

In [40]:
# Limit the data for testing
#train_raw_lines = train_raw_lines[:8000]
#target_raw_lines = target_raw_lines[:8000]

In [7]:
train_raw_lines[0]

'C-Pyrazin-2-yl-methylamine 1h (10.9 g, 100 mmol) was added into a reaction flask, then 20 mL of trifluoroacetic anhydride was added dropwise slowly within an hour at 0° C. in an ice-water bath.'

In [8]:
target_raw_lines[0]

'ADD C-Pyrazin-2-yl-methylamine 1h (10.9 g, 100 mmol); ADD trifluoroacetic anhydride (20 mL) dropwise at 0° C over an hour.'

In [46]:
encoder_in_data = []
decoder_in_data = []

In [50]:
#Define man enc and dec limits

for p,one_in in enumerate(target_raw_lines):
    sptgt = sp_tgt.encode(one_in)
    if len(sptgt)>128:
        continue
    decoder_in_data.append(sptgt)
    encoder_in_data.append(sp_input.encode(train_raw_lines[p]))
max_dencoder_seq_length = max([len(txt) for txt in decoder_in_data])
print('max dec len:',max_dencoder_seq_length)

max_encoder_seq_length = max([len(txt) for txt in encoder_in_data])
print('max enc len:',max_encoder_seq_length)


max dec len: 128
max enc len: 360


In [51]:
len(encoder_in_data)

15464

In [61]:
#encoder_in_data_p = np.zeros((len(train_raw_lines), max_encoder_seq_length), dtype='int')
decoder_in_data_p = np.zeros((len(encoder_in_data), max_dencoder_seq_length+1), dtype='int')
decoder_target_data_p = np.zeros((len(encoder_in_data), 8000), dtype='int')

encoder_in_data_p = tf.ragged.constant(encoder_in_data)


decoder_in_data_r = []
for nu, current_tgt in enumerate(decoder_in_data):
    decoder_in_data_r.append([2])
    for word_idx, word_id in enumerate(current_tgt):
        decoder_in_data_r[nu].append(word_id)
        decoder_in_data_p[nu][0] = 2
        
        decoder_in_data_p[nu][word_idx+1] = word_id
        decoder_target_data_p[nu][word_id] = 1

decoder_in_data_r = tf.ragged.constant(decoder_in_data_r)

In [62]:
print(encoder_in_data_p.shape)
print(decoder_in_data_r.shape)
print(decoder_target_data_p.shape)

(15464, None)
(15464, None)
(15464, 8000)


In [60]:
decoder_target_data_p[0][0]

[0 0 0 ... 0 0 0]


In [14]:
latent_dim = 32
num_encoder_tokens = 8001
num_decoder_tokens = 8000

In [72]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim)
encoder = LSTM(int(latent_dim), return_state=True)

encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(enc_emb(encoder_inputs))

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(num_decoder_tokens, latent_dim,)

decoder_lstm = LSTM(int(latent_dim), return_sequences=False, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb(decoder_inputs),
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [73]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [74]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_10 (Embedding)       (None, None, 32)     256032      ['input_15[0][0]']               
                                                                                                  
 input_16 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 bidirectional_5 (Bidirectional  [(None, 32),        6272        ['embedding_10[0][0]']           
 )                               (None, 16),                                               

In [75]:
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [ ]:
# Train

In [ ]:
model.fit([encoder_in_data_p, decoder_in_data_p], decoder_target_data_p, batch_size = 32, epochs=20)

In [77]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    dec_emb(decoder_inputs), initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [ ]:
decoder_model.summary()

In [94]:
def decode_sequence(input_seq):
     
    
    input_seq = sp_input.encode(input_seq)

    
    states_value = encoder_model.predict([input_seq])

    target_seq = np.zeros((1, 1))

    target_seq[0] = [2]


    stop_condition = False
    decoded_sentence = ''
    decoded_sentence_is = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        #print(output_tokens[0, -1, :])
        # Sample a token
        print(output_tokens)
        sampled_token_index = np.argmax(output_tokens[0])
        decoded_sentence_is.append(int(sampled_token_index))
        
        sampled_char = sp_tgt.decode([int(sampled_token_index)])
        
        decoded_sentence += sampled_char
        print(decoded_sentence)

        if (sampled_token_index == 3 or
           len(decoded_sentence_is) > 37):
            stop_condition = True


        target_seq = np.zeros((1, 1))
        target_seq[0] = [sampled_token_index]

        states_value = [h, c]

    return sp_tgt.decode(decoded_sentence_is)

In [ ]:
decode_sequence('The residue was purified by flash chromatography (1% to 100% EtOAc/Hexanes) to afford methyl 4-(phenylamino)benzoate (961 mg, 100%).')

In [ ]:
valid_raw_loc = 'src-valid.txt'
with open(valid_raw_loc, 'r', encoding='utf-8') as f:
    valid_raw_loc = f.read().split('\n')
    valid_raw_loc = valid_raw_loc[:10]
with open('tgt-valid.txt', 'r', encoding='utf-8') as f:
    valid_raw_tgt = f.read().split('\n')
    valid_raw_tgt = valid_raw_tgt[:10]

for cc,lin in enumerate(valid_raw_loc):
    print(decode_sequence(lin))
    print(valid_raw_tgt[cc])
    print('\n')